1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
N=15000
data = pd.read_csv('data/data_000637.txt', nrows=N)

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [26]:
x = data['BX_COUNTER'].max()
print("Estimated X value: ", x)

Estimated X value:  3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [40]:
start_time = data.loc[0]['TDC_MEAS']*(25/30)+data.loc[0]['BX_COUNTER']*25+data.loc[0]['ORBIT_CNT']*x*25
#Calculate the starting time
abs_time = [0]
for i in range(1,N):
    curr_time = data.loc[i]['TDC_MEAS']*(25/30)+data.loc[i]['BX_COUNTER']*25+data.loc[i]['ORBIT_CNT']*x*25
    #Calculating the time of that misuration
    abs_time.append(curr_time-start_time)
    #Adding the current time in relation to the starting one
data['ABS_TIME'] = pd.to_datetime(abs_time, unit="ns").strftime("%M:%S.%f")
print(data.loc[6])

HEAD                      1
FPGA                      0
TDC_CHANNEL              61
ORBIT_CNT        3869200167
BX_COUNTER             2772
TDC_MEAS                 14
ABS_TIME       00:00.000009
Name: 6, dtype: object


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [41]:
print("Total time to get", N, "misurations:", data['ABS_TIME'].loc[N-1])

Total time to get 15000 misurations: 00:00.013301


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [42]:
top_entries = data.groupby('TDC_CHANNEL').size().nlargest(3)
print("The 3 channels more noisy are:\n", top_entries)

The 3 channels more noisy are:
 TDC_CHANNEL
139    1268
64      752
63      749
dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [13]:
non_empty = data['ORBIT_CNT'].nunique()
print("Number of non empty orbits:", non_empty)

Number of non empty orbits: 150


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [28]:
orbits_139 = data[data['TDC_CHANNEL'] == 139]['ORBIT_CNT'].nunique()
#filtering data for TDC_CHANNEL=139, taking the ORBIT_CNT and counting how many different elements
print("Unique orbits with TDC_CHANNEL=139:", orbits_139)

Unique orbits with TDC_CHANNEL=139: 150


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [34]:
fpga0 = data[data['FPGA'] == 0]['TDC_CHANNEL'].value_counts().sort_index()
fpga1 = data[data['FPGA'] == 1]['TDC_CHANNEL'].value_counts().sort_index()
#Filtering data for the right FPGA, taking only the TDC_CHANNEL coloumn and counting the frequency of each different element
print("FPGA 0:\n", fpga0, "\nFPGA 1:\n", fpga1)

FPGA 0:
 TDC_CHANNEL
1        8
2       16
3       16
4       19
5       19
      ... 
121     63
122     71
123    202
124    193
139    879
Name: count, Length: 117, dtype: int64 
FPGA 1:
 TDC_CHANNEL
1      338
2      363
3      277
4      290
5      189
      ... 
125      6
126      7
127     16
128     18
139    389
Name: count, Length: 124, dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.